In [1]:
import utils
import numpy as np
import os
import pandas as pd
import matplotlib.pyplot as plt
from scipy.stats import wasserstein_distance

pd.set_option('display.expand_frame_repr', False)

%matplotlib inline

In [ ]:
path = "results/reward_vector/"
for file in os.listdir(path):
    if os.path.isfile(path + file):
        shape = np.load(path + file, allow_pickle=True).shape
        print(f"{file:<60}     {shape}")

# Scalar Utilities

In [2]:
path = "results/utilities/"

results = utils.load_results(path)
keys = sorted(results.keys(), key=utils.sorting)

In [ ]:
intra_wds = dict()
for key in keys:
    intra_wds[key] = [wasserstein_distance(results[key][i], results[key][j])
                      for i in range(len(results[key])) for j in range(i+1, len(results[key]))]

inter_wds = dict()
for i in range(len(keys)):
    for j in range(i+1, len(keys)):
        key_1, key_2 = keys[i], keys[j]
        if key_1.split("-")[0] == key_2.split("-")[0]:
            inter_wds[key_1 + "--" + key_2] = [wasserstein_distance(result_1, result_2)
                                            for result_1 in results[key_1] for result_2 in results[key_2]]

for key in keys:
    mean_WD = np.round(np.mean(intra_wds[key]), 2)
    print(f"{key:<40}     {mean_WD}")

In [ ]:
for key in inter_wds.keys():
    mean_WD = np.round(np.mean(inter_wds[key]), 2)
    print(f"{key:<70}     {mean_WD}")

In [3]:
utilities_table = utils.utilities_table(results)

In [4]:
print(utilities_table.pivot(index="Model", columns="Environment", values="Utility"))

                 Environment                    Model Uniques          Utility
0            cum_rewards_env         128_128_64_model     NaN  14.95  (+-0.28)
1            cum_rewards_env  256_128_128_64_64_model     NaN  18.52  (+-0.24)
2            cum_rewards_env              64_64_model     NaN   11.82  (+-0.4)
3          cum_threshold_env         128_128_64_model     NaN   8.47  (+-0.42)
4          cum_threshold_env  256_128_128_64_64_model     NaN   9.28  (+-0.63)
5          cum_threshold_env              64_64_model     NaN   7.55  (+-0.63)
6   cum_threshold_env_linear         128_128_64_model     NaN   9.74  (+-0.43)
7   cum_threshold_env_linear  256_128_128_64_64_model     NaN  10.88  (+-0.39)
8   cum_threshold_env_linear              64_64_model     NaN   9.27  (+-0.53)
9            replication_env         128_128_64_model     NaN  14.35  (+-0.18)
10           replication_env  256_128_128_64_64_model     NaN  19.39  (+-0.29)
11           replication_env              64_64_mode

In [ ]:
print("{:^30}            {:^12}      {:^10}".format("Model", "Mean", "Standard Error"))
for key in keys:
    res = [np.mean(result) for result in results[key]]
    mean = np.round(np.mean(res), 2)
    std_err = np.round(np.std(res)/np.sqrt(len(res)), 2)
    print(f"{key:<40}     {str(mean):<12}       {str(std_err):<30}")

In [ ]:
for key in keys:
    result_list = [np.mean(result) for result in results[key]]
    print(f"{key:<40} {result_list}")

# Fixed Environment Evaluation

In [2]:
path = "results/fixed_env_results/"

results = utils.load_results(path)
keys = sorted(results.keys(), key=utils.sorting)

unique_table = utils.fixed_env_table(results)

In [3]:
print(unique_table.pivot(index="Model", columns="Environment", values="Uniques"))

Environment              replication_env  threshold_env  cum_rewards_env  cum_threshold_env
Model                                                                                      
64_64_model                        141.2          154.2            192.7              194.2
128_128_64_model                   198.8          281.8            222.2              234.7
256_128_128_64_64_model            237.8          440.8            258.5              403.2


In [ ]:
for key in keys:
    uniques = [np.unique(result, axis=0).shape[0] for result in results[key]]
    mean_uniques = np.round(np.mean(uniques), 1)
    print(f"{key:<60} {str(uniques):<40} {mean_uniques}")

# Stale Analysis

In [ ]:
path = "results/legacy_results/"

results_files = {"david": ["david_200221.npy", "david_210221.npy", "david_210221_2.npy"],
                 "my_env": ["my_210220.npy", "my_210221.npy", "my_210221_2.npy"],
                 "new": ["new_gath_repl_1.npy", "new_gath_repl_2.npy", "new_gath_repl_3.npy"],
                 "tfa": ["gathering_replication_1.npy", "gathering_replication_2.npy", "gathering_replication_3.npy",
                         "gathering_replication_4.npy"]


keys = sorted(results_files.keys())
results = dict()
for key in keys:
    results[key] = [np.load(path + file, allow_pickle=True) for file in results_files[key]]

if True:
    for key in keys:
        results[key] = [result[result != -1] for result in results[key]]


In [ ]:
'''
plt.figure(figsize=(15, 12))
i = 1
for result in results["my_env"] + results["david"] + results["tfa"][:-1]:
    plt.subplot(3, 3, i)
    plt.title(f"Number of sampled episodes: {result.shape[0]}")
    plt.hist(result, bins=200, density=True)
    i += 1

plt.tight_layout()
plt.show()
'''
zz =0